<a href="https://colab.research.google.com/github/jonathannocek/yahoo-finance-stocks/blob/master/src/yahoo_stocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Yahoo Stock Data**

### **Part 1: Setup**

In [ ]:
# Install packages
!pip install requests_html
!pip install yahoo_fin
!pip install pandas_datareader

# Imports
import requests
import pandas as pd
import datetime
import numpy
from yahoo_fin import stock_info as si
from pandas_datareader import DataReader
from google.colab import drive

# Mount Google Drive
drive.mount('drive')

# Set Constants
SP_500_TICKERS = si.tickers_sp500()
BASE_URL_1 = 'https://query2.finance.yahoo.com/v10/finance/quoteSummary/'
BASE_URL_2 = '?formatted=true&crumb=swg7qs5y9UP&lang=en-US&region=US&' \
              'modules=defaultKeyStatistics,recommendationTrend,summaryDetail,' \
              'financialData&' \
              'corsDomain=finance.yahoo.com'

### **Part 2: Create dataframe**


**Gather Financial Data**

In [ ]:
financials = []

for ticker in SP_500_TICKERS:
    # Get url and make request
    url = BASE_URL_1 + ticker + BASE_URL_2
    response = requests.get(url)
    response_json = response.json()

    # returns true if status_code less than 400
    if not response.ok:
      row = [ticker, None, None, None, None, None, None, None, None]   
    try:
      results_json = response_json['quoteSummary']['result'][0]
      price = results_json['financialData']['currentPrice']["fmt"]
      shares_outstanding = results_json['defaultKeyStatistics']['sharesOutstanding']["fmt"]
      market_cap = results_json['summaryDetail']['marketCap']['fmt']
      total_cash = results_json['financialData']['totalCash']['fmt']
      total_debt = results_json['financialData']['totalDebt']['fmt']
      total_revenue = results_json['financialData']['totalRevenue']['fmt']
      enterprise_value = results_json['defaultKeyStatistics']['enterpriseValue']['fmt']
      forward_pe = results_json['defaultKeyStatistics']['forwardPE']['fmt']


      row = [ticker, price, shares_outstanding, market_cap, total_cash, total_debt, total_revenue, enterprise_value, forward_pe]

    except Exception as e:
      row = [ticker, None, None, None, None, None, None, None, None]
    
    financials.append(row)

# Create pandas dataframe and insert data   
my_columns = ['Company', 'Price', 'S/O', 'Market Cap', 'Cash', 'Debt', 'Revenue', 'EV', 'Forward PE']
df = pd.DataFrame(financials, columns=my_columns) 
df = df.set_index('Company')

In [43]:
df.head(10).transpose()

Company,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE
Price,98.30,12.16,156.11,497.48,94.86,98.15,307.57,102.40,237.49,473.22
S/O,308.78M,508.11M,69.14M,4.28B,1.76B,204.14M,45.05M,1.77B,636.2M,479.66M
Market Cap,30.35B,6.18B,10.79B,2.13T,167.41B,20.04B,13.86B,181.3B,151.3B,226.99B
Cash,1.36B,9.81B,1.14B,93.03B,6.05B,3.42B,427.36M,5.04B,6.45B,4.35B
Debt,2.32B,40.05B,3.71B,122.19B,87.48B,4.66B,12.12M,19.68B,3.51B,4.7B
Revenue,5.22B,33.36B,9.62B,273.86B,36.23B,186.29B,798.07M,31.44B,44.55B,12.05B
EV,31.34B,36.42B,13.29B,2.16T,248.86B,21.39B,13.44B,196.16B,148.63B,227.33B
Forward PE,26.78,-4.44,17.36,32.01,7.83,11.66,58.14,26.32,29.32,42.59


In [37]:
df.describe()

,Price,S/O,Market Cap,Cash,Debt,Revenue,EV,Forward PE
count,435,435,435,435,434,435,435,435
unique,428,428,418,380,398,406,423,410
top,38.16,1.43B,17.29B,1.14B,3.71B,5.41B,29.25B,11.50
freq,2,4,3,5,3,3,2,3


**Get Analyst Ratings**

In [42]:
ratings = []

for ticker in SP_500_TICKERS:
    # Get url and make request
    url = BASE_URL_1 + ticker + BASE_URL_2
    response = requests.get(url)
    response_json = response.json()

    try:
      # returns true if status_code less than 400
      if not response.ok:
        row = [ticker, None, None]

      # Parse Response
      results_json = response_json['quoteSummary']['result'][0]
      current_month = results_json['recommendationTrend']['trend'][0]

      # Get ratings
      strong_buy = current_month['strongBuy']
      buy = current_month['buy']
      hold = current_month['hold']
      sell = current_month['sell']
      strong_sell = current_month['strongSell']

      # Calculate average
      total_ratings = current_month['strongBuy'] + current_month['buy'] + current_month['hold'] + current_month['sell'] + current_month['strongSell']
      total_score = current_month['strongBuy'] * 1 + current_month['buy'] * 2 + current_month['hold'] * 3 + current_month['sell'] * 4 + current_month['strongSell'] * 5 
      average_score = round(total_score / total_ratings, 2)

      row = [ticker, average_score, total_ratings]
      ratings.append(row)

    except Exception as e:
      row = [ticker, None, None]

# Create dataframe and insert data
my_columns = ['Company', 'Average Score', 'Total Ratings']
df2 = pd.DataFrame(ratings, columns=my_columns) 
df2 = df2.set_index('Company')


In [70]:
df2.head(10).transpose()

Company,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE
Average Score,1.71,2.39,2.56,1.87,2.3,2.36,1.7,2.0,2.24,1.84
Total Ratings,14.00,18.00,27.00,38.00,20.0,14.00,10.0,20.0,29.00,31.00


In [48]:
df2.describe().round(2)

,Average Score,Total Ratings
count,458.00,458.00
mean,2.35,20.19
std,0.41,7.81
min,1.36,1.00
25%,2.06,15.00
50%,2.33,19.00
75%,2.62,25.00
max,5.00,47.00


**Combine Dataframes**

In [57]:
df_new = df.copy()
df_new['Average Score'] = df2['Average Score']
df_new['Total Ratings'] = df2['Total Ratings']
df_new.head(10).transpose()

Company,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE
Price,98.30,12.16,156.11,497.48,94.86,98.15,307.57,102.40,237.49,473.22
S/O,308.78M,508.11M,69.14M,4.28B,1.76B,204.14M,45.05M,1.77B,636.2M,479.66M
Market Cap,30.35B,6.18B,10.79B,2.13T,167.41B,20.04B,13.86B,181.3B,151.3B,226.99B
Cash,1.36B,9.81B,1.14B,93.03B,6.05B,3.42B,427.36M,5.04B,6.45B,4.35B
Debt,2.32B,40.05B,3.71B,122.19B,87.48B,4.66B,12.12M,19.68B,3.51B,4.7B
Revenue,5.22B,33.36B,9.62B,273.86B,36.23B,186.29B,798.07M,31.44B,44.55B,12.05B
EV,31.34B,36.42B,13.29B,2.16T,248.86B,21.39B,13.44B,196.16B,148.63B,227.33B
Forward PE,26.78,-4.44,17.36,32.01,7.83,11.66,58.14,26.32,29.32,42.59
Average Score,1.71,2.39,2.56,1.87,2.3,2.36,1.7,2,2.24,1.84
Total Ratings,14,18,27,38,20,14,10,20,29,31


In [63]:
df_new.describe()

,Average Score,Total Ratings
count,458.000000,458.000000
mean,2.351026,20.189956
std,0.405377,7.806253
min,1.360000,1.000000
25%,2.060000,15.000000
50%,2.330000,19.000000
75%,2.620000,25.000000
max,5.000000,47.000000


**Data Cleaning**

In [58]:
df_new.isna().sum()

Price             8
S/O               8
Market Cap        8
Cash              8
Debt             10
Revenue           8
EV                8
Forward PE        8
Average Score    47
Total Ratings    47
dtype: int64

In [65]:
df_clean = df_new.dropna()

In [66]:
df_clean.describe()

,Average Score,Total Ratings
count,452.000000,452.000000
mean,2.354049,20.159292
std,0.405708,7.711904
min,1.360000,1.000000
25%,2.060000,15.000000
50%,2.335000,19.000000
75%,2.620000,25.000000
max,5.000000,47.000000


In [71]:
df_clean.head(10).transpose()

Company,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE
Price,98.30,12.16,156.11,497.48,94.86,98.15,307.57,102.40,237.49,473.22
S/O,308.78M,508.11M,69.14M,4.28B,1.76B,204.14M,45.05M,1.77B,636.2M,479.66M
Market Cap,30.35B,6.18B,10.79B,2.13T,167.41B,20.04B,13.86B,181.3B,151.3B,226.99B
Cash,1.36B,9.81B,1.14B,93.03B,6.05B,3.42B,427.36M,5.04B,6.45B,4.35B
Debt,2.32B,40.05B,3.71B,122.19B,87.48B,4.66B,12.12M,19.68B,3.51B,4.7B
Revenue,5.22B,33.36B,9.62B,273.86B,36.23B,186.29B,798.07M,31.44B,44.55B,12.05B
EV,31.34B,36.42B,13.29B,2.16T,248.86B,21.39B,13.44B,196.16B,148.63B,227.33B
Forward PE,26.78,-4.44,17.36,32.01,7.83,11.66,58.14,26.32,29.32,42.59
Average Score,1.71,2.39,2.56,1.87,2.3,2.36,1.7,2,2.24,1.84
Total Ratings,14,18,27,38,20,14,10,20,29,31


**TODO: Turn into numerics**

In [72]:
df_final = df_clean.copy()

### **Part 3: Basic Analysis**

Top 10 Stocks to Buy in August 2020 according to Yahoo Finance. 

Note: Some have small sample size - Total Ratings

In [73]:
# Get 10 stocks with the best ratings
df_final.sort_values(by="Average Score").head(10).transpose()

Company,ALGN,BIO,TMO,LKQ,GOOG,LDOS,DAL,AME,CMCSA,FLT
Price,294.29,517.95,428.57,30.92,"1,580.42",89.17,27.27,100.24,43.07,238.67
S/O,78.79M,24.59M,395.59M,304.29M,333.63M,142.19M,637.86M,229.63M,4.56B,84.05M
Market Cap,23.19B,15.39B,169.54B,9.41B,1.07T,12.68B,17.39B,23.02B,196.75B,20.06B
Cash,404.36M,1.03B,5.82B,476.43M,121.08B,590M,15.67B,1.13B,13.93B,765.79M
Debt,71.48M,645.78M,21.35B,4.61B,16.14B,5.66B,34.38B,3.04B,109.98B,4.45B
Revenue,2.16B,2.29B,26.25B,11.78B,166.03B,11.59B,34.06B,4.8B,105.55B,2.57B
EV,22.85B,14.99B,185.07B,13.59B,970.01B,17.75B,36.04B,24.93B,295.22B,23.74B
Forward PE,42.84,51.23,25.85,13.05,28.05,14.11,14.20,24.04,14.55,18.54
Average Score,1.36,1.5,1.53,1.57,1.6,1.6,1.61,1.64,1.65,1.67
Total Ratings,11,4,15,14,5,10,18,14,31,15


In [74]:
# Get 10 stocks with the worst ratings 
df_final.sort_values(by="Average Score", ascending=False).head(10).transpose()

Company,CTVA,ED,UA,VRSN,UAA,CPB,PBCT,TWTR,BEN,GWW
Price,28.44,72.20,8.77,206.73,9.73,52.54,10.45,39.26,20.99,352.61
S/O,748.48M,334.5M,231.5M,114.85M,188.53M,302.16M,424.78M,784.63M,495.36M,53.57M
Market Cap,21.29B,24.15B,4.2B,23.74B,4.2B,15.88B,4.44B,30.8B,10.4B,18.89B
Cash,2.87B,1.14B,1.08B,1.19B,1.08B,1.24B,2.47B,7.77B,6.4B,1.6B
Debt,2.63B,23.95B,2.28B,1.79B,2.28B,6.95B,3.3B,4.13B,997.8M,3.34B
Revenue,14.04B,12.27B,4.51B,1.25B,4.51B,8.36B,1.84B,3.32B,5.39B,11.63B
EV,21.29B,47.18B,5.19B,24.34B,5.62B,21.59B,5.45B,27.33B,6.6B,20.85B
Forward PE,17.34,15.97,58.47,35.34,97.30,18.37,10.89,53.78,8.20,19.67
Average Score,5,3.36,3.33,3.25,3.24,3.24,3.18,3.17,3.15,3.12
Total Ratings,1,14,6,4,33,17,11,36,13,24


### **Part 3: Save to CSV**

In [79]:
# Save files
now = datetime.datetime.now()
df.to_csv(f"yahoo_stock_data_{now.year}_{now.month}_{now.day}_old.csv")
df_current_month.to_csv(f"yahoo_stock_data_{now.year}_{now.month}_{now.day}_current.csv")
